In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [3]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [4]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [7]:
embedding_dim=300
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    inp=Input(shape=(max_len,))

    embed1=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    lstm=LSTM(units=128, dropout=0.3,recurrent_dropout=0.3)(embed1)
    outp=Dense(3,activation='softmax')(lstm)
    model=Model(inputs=inp,outputs=outp)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision,Recall])
    return model

Chạy mô hình với data train

In [8]:
model=model1()
model.summary()
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 50, 300)        │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       219,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,220,035 (23.73 MB)

 Trainable params: 6,220,035 (23.73 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 25s 66ms/step - accuracy: 0.5130 - loss: 0.5719 - precision_1: 0.6046 - recall_1: 0.2477 - val_accuracy: 0.7084 - val_loss: 0.4165 - val_precision_1: 0.7349 - val_recall_1: 0.6784
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step - accuracy: 0.7753 - loss: 0.3478 - precision_1: 0.7969 - recall_1: 0.7469 - val_accuracy: 0.7117 - val_loss: 0.4139 - val_precision_1: 0.7323 - val_recall_1: 0.6704
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 24s 71ms/step - accuracy: 0.8476 - loss: 0.2538 - precision_1: 0.8612 - recall_1: 0.8325 - val_accuracy: 0.7175 - val_loss: 0.4346 - val_precision_1: 0.7304 - val_recall_1: 0.6953
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.8831 - loss: 0.2018 - precision_1: 0.8928 - recall_1: 0.8707 - val_accuracy: 0.7066 - val_loss: 0.5000 - val_precision_1: 0.7178 - val_recall_1: 0.6907
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.9136 - loss: 0.1567 - precision_1: 0.9205 - recall_1

Thử dữ liệu với data test

In [9]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
              precision    recall  f1-score   support

           0       0.63      0.67      0.65      1001
           1       0.63      0.60      0.62      1430
           2       0.71      0.71      0.71      1103

    accuracy                           0.66      3534
   macro avg       0.66      0.66      0.66      3534
weighted avg       0.66      0.66      0.66      3534

